In [1]:
from masters_degree_project.adapters.llm_adapter import LLMAdapter

llm_adapter = LLMAdapter(base_url="http://localhost:8085/v1", api_key="masters_degree_project",
                         model_name="gpt-3.5-turbo")
import os
os.environ['OPENAI_API_KEY'] = "masters_degree_project"
os.environ['OPENAI_API_BASE'] = "http://localhost:8085/v1"
os.environ['USER_AGENT'] = 'myagent'

In [2]:
from masters_degree_project.domain.sentence_decomposition_output import SentenceDecompositionOutput

sentence_decomposition_instruction_template = "Decompose the same sentence into: entities (only nouns and noun phrases), entity_modifiers (Adjectives), behaviors (only verbs and verb phrases), and behaviour_modifiers (adverbs and related phrases), and main clause:{}"

input_sentence = "The quick brown fox jumps over the lazy dog."
sentence_decomposition_output = llm_adapter.generate_instruct_result(
    sentence_decomposition_instruction_template.format(input_sentence),
    SentenceDecompositionOutput)
sentence_decomposition_output

SentenceDecompositionOutput(entity_modifier_pairs=[EntityModifiersPair(entity='fox', entity_modifiers=['quick', 'brown']), EntityModifiersPair(entity='dog', entity_modifiers=['lazy'])], behavior_modifier_pairs=[BehaviorModifiersPair(behavior='jumps over', behavior_modifiers=['over'])], main_clause='The quick brown fox jumps over the lazy dog.')

In [3]:
input_sentences = ["The quick brown fox jumps over the lazy dog.",
                   "Some people like to eat ice cream.",
                   "The sun rises in the east.",
                   "The cat sat on the mat.",
                   "It is illegal to drive a car without a license.",
                   "Is it legal to drive a car without a license?"]
sentence_decomposition_outputs = [
    llm_adapter.generate_instruct_result(sentence_decomposition_instruction_template.format(input_sentence),
                                         SentenceDecompositionOutput) for input_sentence in input_sentences]
sentence_decomposition_outputs

[SentenceDecompositionOutput(entity_modifier_pairs=[EntityModifiersPair(entity='fox', entity_modifiers=['quick', 'brown']), EntityModifiersPair(entity='dog', entity_modifiers=['lazy'])], behavior_modifier_pairs=[BehaviorModifiersPair(behavior='jumps over', behavior_modifiers=['over'])], main_clause='The quick brown fox jumps over the lazy dog.'),
 SentenceDecompositionOutput(entity_modifier_pairs=[EntityModifiersPair(entity='people', entity_modifiers=['some'])], behavior_modifier_pairs=[BehaviorModifiersPair(behavior='like to eat', behavior_modifiers=['to'])], main_clause='Some people like to eat ice cream.'),
 SentenceDecompositionOutput(entity_modifier_pairs=[EntityModifiersPair(entity='sun', entity_modifiers=['the'])], behavior_modifier_pairs=[BehaviorModifiersPair(behavior='rises', behavior_modifiers=['in the east'])], main_clause='The sun rises in the east.'),
 SentenceDecompositionOutput(entity_modifier_pairs=[EntityModifiersPair(entity='cat', entity_modifiers=['the'])], behavior

In [4]:
main_behaviour_composition_instruction_template = (
    "Choose one behaviour and it's modifiers that is reflected in the main clause and compose a concise main_behaviour that represents the action in main clause."
    "main_clause: {}, behaviour_modifier_pairs: {}")

main_clause = sentence_decomposition_outputs[0].concise_main_clause
behaviour_modifier_pairs = sentence_decomposition_outputs[0].behavior_modifier_pairs

In [5]:
from masters_degree_project.domain.main_behaviour_composition_output import MainBehaviourCompositionOutput

main_behaviour_composition_output = llm_adapter.generate_instruct_result(
    main_behaviour_composition_instruction_template.format(main_clause, behaviour_modifier_pairs),
    MainBehaviourCompositionOutput)
main_behaviour_composition_output

MainBehaviourCompositionOutput(main_behaviour='fox leaps')

In [6]:
main_behaviour_composition_outputs = [
    llm_adapter.generate_instruct_result(
        main_behaviour_composition_instruction_template.format(sentence_decomposition_output.concise_main_clause,
                                                               sentence_decomposition_output.behavior_modifier_pairs),
        MainBehaviourCompositionOutput) for sentence_decomposition_output in sentence_decomposition_outputs]
main_behaviour_composition_outputs

[MainBehaviourCompositionOutput(main_behaviour='fox leaps'),
 MainBehaviourCompositionOutput(main_behaviour='eating ice cream'),
 MainBehaviourCompositionOutput(main_behaviour='eastward rising'),
 MainBehaviourCompositionOutput(main_behaviour='cat positioning'),
 MainBehaviourCompositionOutput(main_behaviour='unlicensed driving'),
 MainBehaviourCompositionOutput(main_behaviour='Driving legality')]

In [7]:
input_sentence = "One of the mitigation strategies that was used in the system was to explain the task and the expected output in the prompt of the LLM to guide the output in a certain direction, but this was not always successful, and the hallucinations and omissions were still present in the output."
sentence_decomposition_output = llm_adapter.generate_instruct_result(
    sentence_decomposition_instruction_template.format(input_sentence),
    SentenceDecompositionOutput)
sentence_decomposition_output

SentenceDecompositionOutput(entity_modifier_pairs=[EntityModifiersPair(entity='mitigation strategies', entity_modifiers=['One']), EntityModifiersPair(entity='system', entity_modifiers=['the'])], behavior_modifier_pairs=[BehaviorModifiersPair(behavior='used in the system', behavior_modifiers=['was']), BehaviorModifiersPair(behavior='explain the task and the expected output', behavior_modifiers=['in the prompt of the LLM to guide the output in a certain direction'])], main_clause='One of the mitigation strategies that was used in the system was to explain the task and the expected output in the prompt of the LLM to guide the output in a certain direction, but this was not often')

In [8]:

main_behaviour_composition_output = llm_adapter.generate_instruct_result(
    main_behaviour_composition_instruction_template.format(sentence_decomposition_output.concise_main_clause, sentence_decomposition_output.behavior_modifier_pairs),
    MainBehaviourCompositionOutput)
main_behaviour_composition_output

MainBehaviourCompositionOutput(main_behaviour='prompt-guided explanation')